### Preprocessing module

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate
from tensorflow.keras.models import Sequential, Model

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Настройка стоп-слов и лемматизатора
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [3]:
def preprocess_text(text):
    text = text.lower() # convert text to lowercase
    text = re.sub(r'\b(u\.s\.|us)\b', 'usa', text, flags=re.IGNORECASE)  # replace "U.S." or "US" with "usa"
    text = re.sub(r'<[^>]+>', '', text)  # remove HTML tags
    text = re.sub(r'&\w+;', '', text)  # remove HTML entities
    text = re.sub(r'[^a-z\s]', '', text)  # remove numbers and punctuation (keep only letters and spaces)
    words = text.split()
    words = [word for word in words if word not in stop_words] # remove stop-words, tokenization
    words = [lemmatizer.lemmatize(word) for word in words] # lemmatization
    return ' '.join(words) # combining words into a string

In [4]:
# Загрузка данных
train_df = pd.read_csv('../data/agn_train.csv')
test_df = pd.read_csv('../data/agn_test.csv')

# Применение предобработки к заголовкам и описаниям
train_df['clean_text'] = (train_df['Title'] + ' ' + train_df['Description']).apply(preprocess_text)
test_df['clean_text'] = (test_df['Title'] + ' ' + test_df['Description']).apply(preprocess_text)

# Токенизация
tokenizer = Tokenizer(num_words=5000)  # Ограничение на количество слов
tokenizer.fit_on_texts(train_df['clean_text'])  # Токенизация по тренировочным данным

# Токенизация заголовков и описаний
X_train_title_seq = tokenizer.texts_to_sequences(train_df['Title'])
X_test_title_seq = tokenizer.texts_to_sequences(test_df['Title'])

X_train_description_seq = tokenizer.texts_to_sequences(train_df['Description'])
X_test_description_seq = tokenizer.texts_to_sequences(test_df['Description'])

# Определение максимальной длины
max_length_titles = max([len(x) for x in X_train_title_seq])
max_length_descriptions = max([len(x) for x in X_train_description_seq])

# Выровнять последовательности
X_train_title_pad = pad_sequences(X_train_title_seq, maxlen=max_length_titles)
X_test_title_pad = pad_sequences(X_test_title_seq, maxlen=max_length_titles)

X_train_description_pad = pad_sequences(X_train_description_seq, maxlen=max_length_descriptions)
X_test_description_pad = pad_sequences(X_test_description_seq, maxlen=max_length_descriptions)

# Определение меток для тренировочных данных
y_train = train_df['Class Index'].values - 1  # Приводим классы к диапазону [0, 1, 2, 3]

# Определение меток для тестовых данных
y_test = test_df['Class Index'].values - 1  # Приводим классы к диапазону [0, 1, 2, 3]

# Загрузка предобученных эмбеддингов GloVe
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Создание матрицы эмбеддингов для слов в токенизаторе
embedding_matrix = np.zeros((5000, 100))  # input_dim=5000, output_dim=100 (размерность векторов GloVe)
for word, i in tokenizer.word_index.items():
    if i < 5000:  # Только слова из топ-5000
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Использование предварительно обученных эмбеддингов (GloVe)
embedding_layer = Embedding(input_dim=5000, 
                            output_dim=100, 
                            weights=[embedding_matrix],  # Передача предобученной матрицы эмбеддингов
                            input_length=max_length_titles, 
                            trainable=False)

### Training module

In [5]:
# Вход для заголовков
title_input = Input(shape=(max_length_titles,), name='title_input')
title_embedding = Embedding(input_dim=5000, output_dim=100, weights=[embedding_matrix], input_length=max_length_titles, trainable=False)(title_input)
title_flatten = Flatten()(title_embedding)

# Вход для описаний
description_input = Input(shape=(max_length_descriptions,), name='description_input')
description_embedding = Embedding(input_dim=5000, output_dim=100, weights=[embedding_matrix], input_length=max_length_descriptions, trainable=False)(description_input)
description_flatten = Flatten()(description_embedding)

# Объединение представлений заголовков и описаний
merged = Concatenate()([title_flatten, description_flatten])

# Полносвязные слои после объединения
dense_1 = Dense(128, activation='relu')(merged)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(4, activation='softmax')(dense_2)  # 4 класса

# Создание модели
model = Model(inputs=[title_input, description_input], outputs=output)

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Обучение модели с раздельной обработкой заголовков и описаний
model.fit([X_train_title_pad, X_train_description_pad], y_train, 
          epochs=5, batch_size=32, 
          validation_data=([X_test_title_pad, X_test_description_pad], y_test))


Epoch 1/5


3750/3750 [==============================] - 23s 6ms/step - loss: 0.3626 - accuracy: 0.8714 - val_loss: 0.3267 - val_accuracy: 0.8817
Epoch 2/5
3750/3750 [==============================] - 21s 6ms/step - loss: 0.2836 - accuracy: 0.8983 - val_loss: 0.3311 - val_accuracy: 0.8825
Epoch 3/5
3750/3750 [==============================] - 21s 6ms/step - loss: 0.2258 - accuracy: 0.9194 - val_loss: 0.3460 - val_accuracy: 0.8842
Epoch 4/5
3750/3750 [==============================] - 21s 6ms/step - loss: 0.1699 - accuracy: 0.9390 - val_loss: 0.3978 - val_accuracy: 0.8816
Epoch 5/5
3750/3750 [==============================] - 23s 6ms/step - loss: 0.1233 - accuracy: 0.9559 - val_loss: 0.4665 - val_accuracy: 0.8793


### Hyperparameter selection module

In [9]:
# # Определение max_length на основе тренировочных данных
# X_train_seq = tokenizer.texts_to_sequences(X_train)
# max_length = max([len(seq) for seq in X_train_seq])

# import optuna
# from tensorflow.keras.optimizers import Adam

# # Определение гиперпараметров с Optuna
# def model_builder(trial):
#     # Гиперпараметры для оптимизации
#     embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 200])
#     dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
#     learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
#     num_units = trial.suggest_int('num_units', 64, 512)

#     # Создание модели
#     model = Sequential()
#     model.add(Embedding(5000, embedding_dim, input_length=max_length))  # Используй существующий max_length
#     model.add(Flatten())
#     model.add(Dense(num_units, activation='relu'))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(4, activation='softmax'))

#     # Компиляция модели с оптимизируемыми параметрами
#     optimizer = Adam(learning_rate=learning_rate)
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#     return model

# # Функция для оценки модели
# def objective(trial):
#     model = model_builder(trial)

#     # Обучение модели с оптимизацией
#     model.fit(X_train_pad, y_train_categorical, validation_split=0.1, epochs=5, batch_size=32, verbose=0)

#     # Оценка точности
#     loss, accuracy = model.evaluate(X_test_pad, y_test_categorical, verbose=0)
#     return accuracy  # Оптимизация по точности

# # Запуск Optuna для поиска гиперпараметров
# study = optuna.create_study(direction='maximize')  # Оптимизация точности
# study.optimize(objective, n_trials=10)  # Количество попыток

# # Печать лучших результатов
# print(study.best_trial)

NameError: name 'X_train' is not defined

In [11]:
import optuna
from tensorflow.keras.optimizers import Adam

# Определение гиперпараметров с Optuna
def model_builder(trial):
    # Гиперпараметры для оптимизации
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    num_units = trial.suggest_int('num_units', 64, 512)

    # Вход для заголовков
    title_input = Input(shape=(max_length_titles,), name='title_input')
    title_embedding = Embedding(input_dim=5000, output_dim=100, 
                                weights=[embedding_matrix], input_length=max_length_titles, trainable=False)(title_input)
    title_flatten = Flatten()(title_embedding)
    
    # Вход для описаний
    description_input = Input(shape=(max_length_descriptions,), name='description_input')
    description_embedding = Embedding(input_dim=5000, output_dim=100, 
                                      weights=[embedding_matrix], input_length=max_length_descriptions, trainable=False)(description_input)
    description_flatten = Flatten()(description_embedding)

    # Объединение
    merged = Concatenate()([title_flatten, description_flatten])
    
    # Полносвязные слои
    dense_1 = Dense(num_units, activation='relu')(merged)
    dropout_1 = Dropout(dropout_rate)(dense_1)
    output = Dense(4, activation='softmax')(dropout_1)  # 4 класса

    # Создание модели
    model = Model(inputs=[title_input, description_input], outputs=output)
    
    # Компиляция модели
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Функция для оценки модели
def objective(trial):
    model = model_builder(trial)

    # Обучение модели с оптимизацией
    model.fit([X_train_title_pad, X_train_description_pad], y_train, 
              validation_split=0.1, epochs=5, batch_size=32, verbose=0)

    # Оценка точности
    loss, accuracy = model.evaluate([X_test_title_pad, X_test_description_pad], y_test, verbose=0)
    return accuracy  # Оптимизация по точности

# Запуск Optuna для поиска гиперпараметров
study = optuna.create_study(direction='maximize')  # Оптимизация точности
study.optimize(objective, n_trials=10)  # Количество попыток

# Печать лучших результатов
print(study.best_trial)



[I 2024-10-19 16:10:05,587] A new study created in memory with name: no-name-ea9bb747-76e9-4571-8fef-92eb0ac3e50f
[I 2024-10-19 16:13:20,784] Trial 0 finished with value: 0.8797368407249451 and parameters: {'dropout_rate': 0.3947280315537505, 'learning_rate': 1.2041911953240719e-05, 'num_units': 256}. Best is trial 0 with value: 0.8797368407249451.
[I 2024-10-19 16:15:37,563] Trial 1 finished with value: 0.8739473819732666 and parameters: {'dropout_rate': 0.30281916346028387, 'learning_rate': 0.00391087068818233, 'num_units': 170}. Best is trial 0 with value: 0.8797368407249451.
[W 2024-10-19 16:17:55,314] Trial 2 failed with parameters: {'dropout_rate': 0.26712349381283707, 'learning_rate': 0.0020540232211003007, 'num_units': 433} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^

KeyboardInterrupt: 